# 2050 Data Loading

This notebook loads all of the Arlington 2050 data from files, does minimal cleaning, and outputs a single Pandas DataFrame that has two columns:

- **concatenated_text**: This combines all of the text from the entire postcard submission. Depending on the specific source dataset, this might involve a lot of cleaning. If the data includes english translations, only the english data is kept.
- **source**: a text column that describes the source of the data. Eg "County Fair"


TODO: Add spell checking with pyspellchecker. I kept the import here so that I can add it later.


In [ ]:
%%bash
pip install ipython ipykernel ipywidgets
pip install spacy
python -m spacy download en_core_web_md
pip install plotly
pip install scikit-learn
pip install pandas
pip install openpyxl
pip install pyspellchecker
pip install nbformat

In [21]:
import pandas as pd
from spellchecker import SpellChecker

# initialize models
spell = SpellChecker()

Load all of the datasets into separate dataframes

In [22]:
# Load the Excel file into a pandas DataFrame
df_countyfair = pd.read_excel('data/Digitized postcards from County Fair_August  23-25 2024.xlsx')
df_hispanicfest = pd.read_excel('data/Digitized postcards from Hispanic Heritage Community Festival in Tyrol Hill Park_Sep 7 2024.xlsx')
df_web = pd.read_excel('data/Public Input Potcards Exported 10-01-24.xlsx')
df_events = pd.read_excel('data/Postcard text tracker.xlsx')

Now clean the data. These spreadsheets are all slightly different, so i'm trying to get them all to the same subset of columns. 

Eventually, I might come back and try to find extra data in individual spreadsheets that might be interesting to include.

In [23]:
###
# County Fair
###
# Drop the first two rows
df_countyfair = df_countyfair.drop([0, 1]).reset_index(drop=True)

# Clean up the columns
df_countyfair.columns = ['id', 'first', 'first_translated', 'second', 'second_translated']

# Create a single column that concatenates the text from the other columns
def concatenate_text(row):
    text = ""
    if pd.notna(row['first_translated']):
        text += row['first_translated']
    elif pd.notna(row['first']):
        text += row['first']
    
    if pd.notna(row['second_translated']):
        text += ". " + row['second_translated']
    elif pd.notna(row['second']):
        text += ". " + row['second'] 

    return text

df_countyfair['concatenated_text'] = df_countyfair.apply(concatenate_text, axis=1)
df_countyfair['source'] = "County Fair"

In [24]:
###
# Hispanic Heritage Fest
###
# Drop the first two rows
df_hispanicfest = df_hispanicfest.drop([0, 1]).reset_index(drop=True)

# Clean up the columns
df_hispanicfest.columns = ['id', 'first', 'first_translated', 'second', 'second_translated']

# Create a single column that concatenates the text from the other columns
def concatenate_text(row):
    text = ""
    if pd.notna(row['first_translated']):
        text += row['first_translated']
    elif pd.notna(row['first']):
        text += row['first']
    
    if pd.notna(row['second_translated']):
        text += ". " + row['second_translated']
    elif pd.notna(row['second']):
        text += ". " + row['second'] 

    return text

df_hispanicfest['concatenated_text'] = df_hispanicfest.apply(concatenate_text, axis=1)
df_hispanicfest['source'] = "Hispanic Heritage Fest"

In [25]:
###
# Web form
###

# Clean up the columns
df_web.columns = ['id', 'zip', 'source', 'first', 'first_gettinghere','second','third','zip_selfreported','zip_selfreported2']

# Create a single column that concatenates the text from the other columns
def concatenate_text(row):
    text = ""
    if pd.notna(row['first']):
        text += row['first']
    if pd.notna(row['first_gettinghere']):
    
        text += row['first_gettinghere']     
    if pd.notna(row['second']):
        text += row['second']
    if pd.notna(row['third']):
        text += row['third']

    return text

df_web['concatenated_text'] = df_web.apply(concatenate_text, axis=1)
df_web['source'] = "web"

In [26]:
###
# Postcards from events
###
# Drop the first two rows
df_events = df_events.dropna(subset=['Postcard #', 'Text'])

# Clean up the columns
df_events.columns = ['id', 'concatenated_text','ignore','source','themes','language','demographics']

In [27]:
#combine all of these into a single dataframe, with concatenated text and source.
df_all_postcards = pd.concat([
        df_countyfair[['concatenated_text','source']]
        ,df_hispanicfest[['concatenated_text','source']]
        ,df_web[['concatenated_text','source']]
        ,df_events[['concatenated_text','source']]
        ])

In [ ]:
print(f"""here's the number of records from each event:
      Hispanic Heritage Fest: {df_hispanicfest.shape[0]}
        County Fair: {df_countyfair.shape[0]}
        Website: {df_web.shape[0]}
        Other Events: {df_events.shape[0]}""")

In [ ]:
df_all_postcards.describe()